In [241]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
#just for the notebook
%run implementations
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [260]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv' 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)


In [261]:
tX

array([[ 138.47 ,   51.655,   97.827, ...,    1.24 ,   -2.475,  113.497],
       [ 160.937,   68.768,  103.235, ..., -999.   , -999.   ,   46.226],
       [-999.   ,  162.172,  125.953, ..., -999.   , -999.   ,   44.251],
       ...,
       [ 105.457,   60.526,   75.839, ..., -999.   , -999.   ,   41.992],
       [  94.951,   19.362,   68.812, ..., -999.   , -999.   ,    0.   ],
       [-999.   ,   72.756,   70.831, ..., -999.   , -999.   ,    0.   ]])

In [262]:
#replace -999 with nan
tX = replace_999_with_nan(tX)
tX

array([[138.47 ,  51.655,  97.827, ...,   1.24 ,  -2.475, 113.497],
       [160.937,  68.768, 103.235, ...,     nan,     nan,  46.226],
       [    nan, 162.172, 125.953, ...,     nan,     nan,  44.251],
       ...,
       [105.457,  60.526,  75.839, ...,     nan,     nan,  41.992],
       [ 94.951,  19.362,  68.812, ...,     nan,     nan,   0.   ],
       [    nan,  72.756,  70.831, ...,     nan,     nan,   0.   ]])

In [257]:
#substract mean and divide by std
tX = normalize(tX)

In [268]:
inds = np.where(np.isnan(tX))

tX[inds]

array([nan, nan, nan, ..., nan, nan, nan])

In [245]:
#Obtain mean of columns as you need, nanmean is just convenient.
col_mean = np.nanmean(tX, axis=0)
print(col_mean)


#Find indicies that you need to replace
inds = np.where(np.isnan(tX))

#Place column means in the indices. Align the arrays using take
tX[inds] = np.take(col_mean, inds[1])

inds = np.where(np.isnan(tX))
inds


[-2.51508626e-02  1.02028164e-16  9.75131087e-18  9.16200449e-17
 -1.20898347e-01 -2.60406414e-01  1.14403083e-01 -7.39790673e-16
 -4.45660397e-16 -9.45576950e-17  3.87708976e-16  5.77852433e-16
 -7.63778263e-03  4.18840296e-16  2.33848496e-17  4.32400782e-17
 -5.84950755e-16 -3.10063086e-18 -8.78408457e-18 -3.76882969e-16
 -2.60067523e-17 -4.27278657e-16  9.79176000e-01 -1.26895413e-01
  7.33351673e-04 -4.54163006e-03 -2.16983065e-01  6.44682656e-04
 -1.63188290e-04  3.71608522e-16]


(array([], dtype=int64), array([], dtype=int64))

In [246]:
x_train, y_train, x_test, y_test = split_data(tX, y, 0.8, seed=1)

In [247]:
np.isnan(y_test).any()

False

In [248]:
# Append y values as column to later divide y into buckets corresponding with x values
train = np.column_stack((x_train, y_train))

In [249]:
# Split training x into buckets
buckets = get_buckets(train)

In [250]:
x_train.shape
initial_weights = np.ones(x_train.shape[1])

In [269]:
from implementations import log_reg
# Least squares
#w , loss = least_squares(y_train,x_train)

# Least squares GD
#w , loss = least_squares_GD(y_train,x_train,initial_weights,200,0.01)

# Ridge regression
w,loss = ridge_regression(y_train,x_train,0.001)

# Logistic regression
#w,loss = log_reg(y_train,x_train,initial_weights,300,1)

# reg logistic regression
#w,loss = reg_logistic_regression(y_train,x_train,0.001,initial_weights,300,1)

In [270]:
w

array([ 1.80847646e-02, -2.49250570e-01, -2.59649680e-01,  3.39065165e-02,
        3.21117036e-02,  2.50253917e-01,  3.61480342e-02,  2.74732259e-01,
       -3.99510065e-02, -1.21071893e+00, -1.86328635e-01,  1.25672144e-01,
        1.24077483e-01,  4.13290101e-01,  2.15460260e-04, -1.21488629e-03,
        5.10619196e-01, -6.39629903e-04,  2.86357982e-03,  9.30359899e-02,
        3.49431665e-04, -4.27359314e-02, -3.41691519e-01, -4.31453221e-01,
       -4.67763366e-04,  4.47027542e-04, -2.56593496e-01,  3.43294573e-03,
       -3.82869377e-03,  1.59079095e+00])

## Do your thing crazy machine learning thing here :) ...

## Generate predictions and save ouput in csv format for submission:

In [103]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [114]:
weights = w
weights.shape

(30,)

In [115]:
OUTPUT_PATH = '../data/sub.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

# Computing score on validation set

In [271]:
weights = w
y_pred = predict_labels(weights,x_test)

In [272]:
w

array([ 1.80847646e-02, -2.49250570e-01, -2.59649680e-01,  3.39065165e-02,
        3.21117036e-02,  2.50253917e-01,  3.61480342e-02,  2.74732259e-01,
       -3.99510065e-02, -1.21071893e+00, -1.86328635e-01,  1.25672144e-01,
        1.24077483e-01,  4.13290101e-01,  2.15460260e-04, -1.21488629e-03,
        5.10619196e-01, -6.39629903e-04,  2.86357982e-03,  9.30359899e-02,
        3.49431665e-04, -4.27359314e-02, -3.41691519e-01, -4.31453221e-01,
       -4.67763366e-04,  4.47027542e-04, -2.56593496e-01,  3.43294573e-03,
       -3.82869377e-03,  1.59079095e+00])

In [273]:
# Compute accuracy 
true_positive = 0
false_positive = 0 
true_negative = 0
false_negative = 0

testing_length = y_test.shape[0]
for i in range(testing_length):      
    if (y_test[i] == y_pred[i]) and (y_pred[i]==1):        
        true_positive += 1
    if (y_test[i] == y_pred[i]) and (y_pred[i]==-1):
        true_negative +=1        
    if (y_test[i] != y_pred[i]) and (y_pred[i]==-1):    
        false_negative += 1
    if (y_test[i] != y_pred[i]) and (y_pred[i]==1):
        false_positive +=1
        
        
precision = true_positive/float(true_positive+false_positive)
recall =true_positive/float(true_positive + false_negative)
F=2 *(precision*recall)/float(precision+recall)        
    

print('Train accuracy: ',  ((true_positive+true_negative)/float(testing_length))*100.0)
print("precision : ", precision)
print("recall : ",recall)
print("F1 score : ",F)

        

Train accuracy:  74.51
precision :  0.6652420602755876
recall :  0.5239698200812536
F1 score :  0.5862147332878802
